In [ ]:
import MySQLdb
from sklearn.linear_model import LogisticRegression

conn = MySQLdb.connect(host='219.224.169.45', user='lizimeng', passwd='codegeass', db='investor', charset='utf8')
cursor = conn.cursor()

# 账号
accounts = []
sql = 'select distinct capital_account_new from chicang'
cursor.execute(sql)
for row in cursor.fetchall():
    accounts.append(row[0])
    
# 日期 (昨天、今天、明天)
days = []
lastdays = []
nextdays = []
sql = "select day,lastday,nextday from commodity.trade_day where day between '20140101' and '20161231' \
order by day asc"
cursor.execute(sql)
for row in cursor.fetchall()[1:-1]:
    days.append(row[0])
    lastdays.append(row[1])
    nextdays.append(row[2])
    
logit = LogisticRegression(fit_intercept=True, n_jobs=-1)

#for account in ['579656682822',]:
for account in accounts:
    X = []
    Y = []
    for i in range(len(days)):
        day = days[i]
        lastday = lastdays[i]
        nextday = nextdays[i]
        #  今日持仓合约 或 明日持仓合约
        sql = "(select distinct vari_code,deliv_date from chicang where capital_account_new=%s and tradedate=%s) union \
        (select distinct vari_code,deliv_date from chicang where capital_account_new=%s and tradedate=%s)"
        cursor.execute(sql, (account,day,account,nextday))
        contracts = cursor.fetchall()
        for contract in contracts:
            # 结算价
            sql = "select settle from commodity.contract_daily where date=%s and code=%s and delivery=%s"
            if cursor.execute(sql, (day,contract[0],contract[1]))==0:
                continue
            settle = cursor.fetchone()[0]
            if cursor.execute(sql, (lastday,contract[0],contract[1]))==0:
                continue
            settle_last = cursor.fetchone()[0]
            if settle==settle_last:
                continue
            x = 1 if settle>settle_last else -1
            # 持仓
            sql = "select sum(qty) from chicang where capital_account_new=%s and tradedate=%s and vari_code=%s and \
            deliv_date=%s and bs_flag=%s"
            for bs_flag in ('0','1'):
                if cursor.execute(sql, (account,day,contract[0],contract[1],bs_flag))>0:
                    oi = cursor.fetchone()[0]
                else:
                    oi = 0
                if cursor.execute(sql, (account,nextday,contract[0],contract[1],bs_flag))>0:
                    oi_next = cursor.fetchone()[0]
                else:
                    oi_next = 0
                if bs_flag=='0' and oi_next>oi:
                    y = 1
                    X.append((x,))
                    Y.append(y)
                elif bs_flag=='0' and oi_next<oi:
                    y = 0
                    X.append((x,))
                    Y.append(y)
                elif bs_flag=='1' and oi_next>oi:
                    y = 0
                    X.append((x,))
                    Y.append(y)
                elif bs_flag=='1' and oi_next<oi:
                    y = 1
                    X.append((x,))
                    Y.append(y)
    if len(X)>0:
        logit.fit(X,Y)
        
        print account
        print 'coef: %f' % logit.coef_
        print logit.classes_
        print logit.predict_proba(((1,),(-1,)))
        print 

cursor.close()
conn.close()